In [1]:
# all codes come are changed based on LDR code.
import argparse
import torch.optim as optim
import os
import yaml
from mcrgan.default import _C as config
from mcrgan.default import update_config
import torch
from mcrgan.trainer import MCRTrainer
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.models import get_BNN_models
from mcrgan.models import get_noise
from mcrgan.models import DiscriminatorMNIST
from mcrgan.models import weights_init_mnist_model_noise

import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os
from mcrgan.loss import MCRGANloss
import random
random.seed(0)

In [2]:
# data_preprocessing and parameters

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
DATASET = 'mnist'
ROOT    = 'pth/to/the/dataset'
BATCH_SIZE = 2048
WORKERS = 0
DATA_SAMPLE = 50000
LABEL = 0

dataloader, dataset = get_dataloader(
        data_name=DATASET,
        root=ROOT,
        batch_size=BATCH_SIZE,
        num_workers=WORKERS)


In [3]:
import collections

def get_netG(netD,netV,net_noise,radius=1):
    models = [netD, netV,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    fed_state_dict = collections.OrderedDict()
    for key in weight_keys:
        fed_state_dict[key] = worker_state_dict[0][key] + radius*worker_state_dict[2][key]*torch.log(1+torch.exp(worker_state_dict[1][key]))
    return fed_state_dict

# new netG already optimized by adam and time learning rate, so we only using netG to update netV
def update_netV(netV,netG,net_noise,fed_state_dict,radius=1):
    models =[netV,netG,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    update_dict = collections.OrderedDict()
    for key in weight_keys:
        update_dict[key] = worker_state_dict[0][key] - (worker_state_dict[1][key]-fed_state_dict[key])*(worker_state_dict[2][key]*radius)/torch.log(1+torch.exp(-worker_state_dict[0][key]))
    return update_dict
        

In [4]:
#models
# train hyperparameters, batch norm not used var
#store_Var = []
import time
from torch_mimicry.training import scheduler, logger, metric_log

from test_acc import nearsub
import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features2
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os

start_time = time.time()
#global_step = 0
num_steps = 150
n_dis = 1
n_dis2 = 1
numclasses =10
lr1 = 1e-3
lr2 = 1e-3
log_data = metric_log.MetricLog()
data_time = time.time()
iter_dataloader = iter(dataloader)
data, label = next(iter_dataloader)
data_time = time.time() - data_time
real_cpu = data.to(device)
real_label = label.clone().detach()

netD, netG = get_models(DATASET, device)

optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))

Acc_results_D = []
Acc_results_G = []




transform = transforms.Compose(
            [transforms.Resize(32),
             transforms.ToTensor(),
             transforms.Normalize(0.5, 0.5)])

trainset = datasets.MNIST(root=ROOT, train=True, download=True, transform=transform)
testset = datasets.MNIST(root=ROOT, train=False, download=True, transform=transform)

testloader = torch.utils.data.DataLoader(
        testset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)

trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
n_comp = 10


In [5]:
netD, netG

(DataParallel(
   (module): DiscriminatorMNIST(
     (main): Sequential(
       (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
       (1): LeakyReLU(negative_slope=0.2, inplace=True)
       (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
       (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (4): LeakyReLU(negative_slope=0.2, inplace=True)
       (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
       (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (7): LeakyReLU(negative_slope=0.2, inplace=True)
       (8): Conv2d(256, 128, kernel_size=(4, 4), stride=(1, 1), bias=False)
       (9): Flatten(start_dim=1, end_dim=-1)
     )
   )
 ),
 DataParallel(
   (module): GeneratorMNIST(
     (main): Sequential(
       (0): ConvTranspose2d(128, 256, kernel_size=(4, 4), stride=(1, 1), bias=False)
       (1)

In [6]:
real_cpu.shape

torch.Size([2048, 1, 32, 32])

In [7]:
netD(real_cpu).shape

torch.Size([2048, 128])

In [8]:
netG(netD(real_cpu)).shape

torch.Size([2048, 1, 32, 32])

In [9]:
netD(netG(netD(real_cpu))).shape

torch.Size([2048, 128])

In [10]:
netD(netG(netD(real_cpu)))

tensor([[-0.0033, -0.0189, -0.0870,  ...,  0.0505,  0.0416,  0.0232],
        [-0.0245, -0.0181, -0.1338,  ..., -0.0436,  0.0787, -0.0667],
        [-0.1540,  0.2556, -0.0298,  ..., -0.0534,  0.1048, -0.0206],
        ...,
        [-0.0724, -0.0055, -0.0238,  ...,  0.0136,  0.0486, -0.0488],
        [-0.0019,  0.0317, -0.0184,  ..., -0.0658, -0.0147, -0.0056],
        [ 0.0527,  0.0086,  0.0670,  ...,  0.0191,  0.0684,  0.0741]],
       device='cuda:0', grad_fn=<DivBackward0>)

In [11]:
# fixed radius case, due to this requires more steps to go to about -200 then set 1500 epoches

n_dis = 1
n_dis2 = 1
mode = 11
gam3 = 1
mcr_gan_loss = MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)

num_steps = 500
lr1 = 1e-3
lr2 = 1e-3
optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
meanV_sum = []
radius_list = []
radius = 0

for epoch in range(num_steps):
    for i in range(n_dis):
        netD.zero_grad()
        optD.zero_grad()
        Z = netD(real_cpu)
        fake_img = netG(netD(real_cpu)).detach()
        Z_bar = netD(fake_img)
        errD, errD_EC = mcr_gan_loss(Z, Z_bar, real_label, 1, n_dis)
        print(errD, errD_EC)
        errD.backward()
        optD.step()
        
    for j in range(n_dis2):
        netG.zero_grad()
        optG.zero_grad()
        
        Z = netD(real_cpu)
        fake_img = netG(netD(real_cpu)).detach()
        Z_bar = netD(fake_img)
        
        errG, errD_EC = mcr_gan_loss(Z, Z_bar, real_label, 1, n_dis2)
        print(errG, errD_EC)
        errG = - errG
        errG.backward()
        optG.step()

    
    

tensor(-20.8329, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-9.2874, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.5539, device='cuda:0', grad_fn=<NegBackward0>), tensor(-53.1404, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-11.8721, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-4.4717, device='cuda:0', grad_fn=<NegBackward0>), tensor(-3.9362, device='cuda:0', grad_fn=<NegBackward0>), tensor(-30.9512, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-11.8721, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-4.4717, device='cuda:0', grad_fn=<NegBackward0>), tensor(-3.9362, device='cuda:0', grad_fn=<NegBackward0>), tensor(-30.9512, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-18.9148, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-7.7846, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.2802, device='cuda:0', grad_fn=<NegBackward0>), tensor(-45.9901, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-18.9148, device='cuda:0', grad_fn=<NegBackward0>) [tenso

tensor(-42.9393, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-21.4593, device='cuda:0', grad_fn=<NegBackward0>), tensor(-17.1574, device='cuda:0', grad_fn=<NegBackward0>), tensor(-74.3854, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-42.9393, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-21.4593, device='cuda:0', grad_fn=<NegBackward0>), tensor(-17.1574, device='cuda:0', grad_fn=<NegBackward0>), tensor(-74.3854, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-44.0896, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-21.9218, device='cuda:0', grad_fn=<NegBackward0>), tensor(-17.8287, device='cuda:0', grad_fn=<NegBackward0>), tensor(-75.6453, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-44.0896, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-21.9218, device='cuda:0', grad_fn=<NegBackward0>), tensor(-17.8287, device='cuda:0', grad_fn=<NegBackward0>), tensor(-75.6452, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-44.1315, device='cuda:0', grad_fn=<NegBackward0>

tensor(-52.7741, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-26.1402, device='cuda:0', grad_fn=<NegBackward0>), tensor(-22.2295, device='cuda:0', grad_fn=<NegBackward0>), tensor(-80.8101, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-53.3796, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-26.4513, device='cuda:0', grad_fn=<NegBackward0>), tensor(-22.5236, device='cuda:0', grad_fn=<NegBackward0>), tensor(-80.8290, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-53.3796, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-26.4513, device='cuda:0', grad_fn=<NegBackward0>), tensor(-22.5236, device='cuda:0', grad_fn=<NegBackward0>), tensor(-80.8290, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-53.0830, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-26.3554, device='cuda:0', grad_fn=<NegBackward0>), tensor(-22.3215, device='cuda:0', grad_fn=<NegBackward0>), tensor(-80.9440, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-53.0830, device='cuda:0', grad_fn=<NegBackward0>

tensor(-58.5337, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-28.8672, device='cuda:0', grad_fn=<NegBackward0>), tensor(-25.2268, device='cuda:0', grad_fn=<NegBackward0>), tensor(-83.7499, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-58.5337, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-28.8672, device='cuda:0', grad_fn=<NegBackward0>), tensor(-25.2268, device='cuda:0', grad_fn=<NegBackward0>), tensor(-83.7499, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-59.7841, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-29.4377, device='cuda:0', grad_fn=<NegBackward0>), tensor(-25.8984, device='cuda:0', grad_fn=<NegBackward0>), tensor(-84.4642, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-59.7842, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-29.4377, device='cuda:0', grad_fn=<NegBackward0>), tensor(-25.8984, device='cuda:0', grad_fn=<NegBackward0>), tensor(-84.4642, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-60.5378, device='cuda:0', grad_fn=<NegBackward0>

tensor(-64.0278, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-31.4966, device='cuda:0', grad_fn=<NegBackward0>), tensor(-28.0560, device='cuda:0', grad_fn=<NegBackward0>), tensor(-86.8155, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-64.1807, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-31.4992, device='cuda:0', grad_fn=<NegBackward0>), tensor(-28.2032, device='cuda:0', grad_fn=<NegBackward0>), tensor(-87.0855, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-64.1807, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-31.4992, device='cuda:0', grad_fn=<NegBackward0>), tensor(-28.2032, device='cuda:0', grad_fn=<NegBackward0>), tensor(-87.0855, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-64.9208, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-31.7694, device='cuda:0', grad_fn=<NegBackward0>), tensor(-28.6731, device='cuda:0', grad_fn=<NegBackward0>), tensor(-87.0859, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-64.9208, device='cuda:0', grad_fn=<NegBackward0>

...

tensor(-93.4776, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-44.9277, device='cuda:0', grad_fn=<NegBackward0>), tensor(-43.9129, device='cuda:0', grad_fn=<NegBackward0>), tensor(-102.2273, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-93.1041, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-44.7195, device='cuda:0', grad_fn=<NegBackward0>), tensor(-43.7491, device='cuda:0', grad_fn=<NegBackward0>), tensor(-102.0854, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-93.1041, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-44.7195, device='cuda:0', grad_fn=<NegBackward0>), tensor(-43.7490, device='cuda:0', grad_fn=<NegBackward0>), tensor(-102.0854, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-93.1942, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-44.7092, device='cuda:0', grad_fn=<NegBackward0>), tensor(-43.8488, device='cuda:0', grad_fn=<NegBackward0>), tensor(-102.1567, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-93.1942, device='cuda:0', grad_fn=<NegBackwa

tensor(-94.3184, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-45.2159, device='cuda:0', grad_fn=<NegBackward0>), tensor(-44.4610, device='cuda:0', grad_fn=<NegBackward0>), tensor(-102.7078, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-94.3184, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-45.2159, device='cuda:0', grad_fn=<NegBackward0>), tensor(-44.4610, device='cuda:0', grad_fn=<NegBackward0>), tensor(-102.7078, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-93.4908, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-44.8885, device='cuda:0', grad_fn=<NegBackward0>), tensor(-43.9648, device='cuda:0', grad_fn=<NegBackward0>), tensor(-102.2897, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-93.4908, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-44.8885, device='cuda:0', grad_fn=<NegBackward0>), tensor(-43.9648, device='cuda:0', grad_fn=<NegBackward0>), tensor(-102.2897, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-92.5001, device='cuda:0', grad_fn=<NegBackwa

tensor(-100.0001, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.6313, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.7016, device='cuda:0', grad_fn=<NegBackward0>), tensor(-105.4035, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-99.8787, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.5684, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.6437, device='cuda:0', grad_fn=<NegBackward0>), tensor(-105.3343, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-99.8787, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.5684, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.6437, device='cuda:0', grad_fn=<NegBackward0>), tensor(-105.3343, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-99.7715, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.5250, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.5808, device='cuda:0', grad_fn=<NegBackward0>), tensor(-105.2375, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-99.7715, device='cuda:0', grad_fn=<NegBackw

tensor(-99.0357, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.2656, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.1065, device='cuda:0', grad_fn=<NegBackward0>), tensor(-105.0213, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-99.0357, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.2656, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.1065, device='cuda:0', grad_fn=<NegBackward0>), tensor(-105.0213, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-99.5883, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.5167, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.4065, device='cuda:0', grad_fn=<NegBackward0>), tensor(-105.1763, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-99.5883, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.5167, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.4065, device='cuda:0', grad_fn=<NegBackward0>), tensor(-105.1763, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-100.3445, device='cuda:0', grad_fn=<NegBackw

tensor(-99.8525, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.7288, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.4564, device='cuda:0', grad_fn=<NegBackward0>), tensor(-105.4146, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-100.1536, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.8381, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.6477, device='cuda:0', grad_fn=<NegBackward0>), tensor(-105.4659, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-100.1536, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.8381, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.6477, device='cuda:0', grad_fn=<NegBackward0>), tensor(-105.4659, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-100.1452, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-47.8736, device='cuda:0', grad_fn=<NegBackward0>), tensor(-47.6035, device='cuda:0', grad_fn=<NegBackward0>), tensor(-105.5034, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-100.1452, device='cuda:0', grad_fn=<NegBa

In [12]:
#acc at first for NetD
NetD_acc = 0
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features(testloader, netD, netG)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features(testloader, netD, netG)


print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

#acc at first for NetD
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
        
NetD_acc = count/10000
print('NetD, Acc at first', NetD_acc)
Acc_results_D.append(count/10000)

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.18s/it]


----------------------------
Train_z, Test_z
PCA: 0.9572
SVD: 0.9561
NetD, Acc at first 0.9725
